In [1]:
deployment = "gpt4"
model = "gpt-4"

# 回顾

In [5]:
from langchain import PromptTemplate, OpenAI, LLMChain
from langchain.chat_models import AzureChatOpenAI
from langchain.utilities import GoogleSerperAPIWrapper
from langchain.chat_models import AzureChatOpenAI
# from langchain.chat_models import ChatOpenAI #直接访问OpenAI的GPT服务

#llm = ChatOpenAI(model_name="gpt-4", temperature=0) #直接访问OpenAI的GPT服务
llm = AzureChatOpenAI(deployment_name = deployment, model_name=model, temperature=0, max_tokens=200) #通过Azure的OpenAI服务

# import os
# os.environ["SERPER_API_KEY"] = ""
# https://serper.dev

def query_web(question):
    search = GoogleSerperAPIWrapper()
    return search.run(question)

In [33]:

print(query_web("what date is today?"))

Saturday, September 9, 2023


In [8]:
def get_current_cluster_state(cluster_name):
    return  """ERROR: Failed to pull image "chaocai/docker/dsp:latest"""

In [6]:
funcs = {
    "get_current_cluster_state": get_current_cluster_state,
    "query_web":query_web,
}

In [29]:
import openai
import json
def run(input):
    msg=[{"role":"user","content":input}]
    ret = run_conversation(msg)
    return ret["content"] 

def run_conversation(msg):
    response = openai.ChatCompletion.create(
        engine=deployment,
        model=model,
        messages=msg,
        functions=[
     
            {
                "name": "get_current_cluster_state",
                "description": "Get the current state in a given cluster",
                "parameters": {
                    "type": "object",
                    "properties": {
                        "cluster_name": {
                            "type": "string",
                            "description": "the name of the cluster",
                        },
                        
                    },
                    "required": ["cluster_name"],
                },
            },
     
            {
                "name": "query_web",
                "description": "Get a realtime info/unknown info from web",
                "parameters": {
                    "type": "object",
                    "properties": {
                        "question": {
                            "type": "string",
                            "description": "the query/question",
                        },
                        
                    },
                    "required": ["query"],
                },
            }
        ],
        function_call="auto",
    )

    message =  response["choices"][0]["message"]
    
    # 如果不需要调用function，则直接返回结果
    if not message.get("function_call"):
        return message
    
    # 从
    function_name = message["function_call"]["name"]
    function_args = json.loads(message["function_call"]["arguments"])
    print(function_args)
    res = funcs[function_name](function_args["question"])
    message["content"]=None
    msg.append(message)
    msg.append({
                "role": "function",
                "name": function_name,
                "content": res,
            })
    return run_conversation(msg)

In [36]:
run("北京今天适合穿什么")

{'question': '北京今天的天气如何'}


'今天北京的天气大约是69°F（约20.5°C），建议您可以穿着轻薄的长袖或者短袖，如果觉得有些凉，可以加一件薄外套。'

# LangChain Agent

In [2]:
from langchain.agents import initialize_agent, Tool
from langchain.chat_models import AzureChatOpenAI
from langchain.chains import LLMRequestsChain
from langchain.chains import LLMChain
from langchain.prompts import PromptTemplate
import time


In [3]:
# from langchain.chat_models import ChatOpenAI #直接访问OpenAI的GPT服务

#llm = ChatOpenAI(model_name="gpt-4", temperature=0) #直接访问OpenAI的GPT服务
llm = AzureChatOpenAI(deployment_name = deployment, model_name=model, temperature=0, max_tokens=200) #通过Azure的OpenAI服务


In [9]:
tools = [
    Tool(
        name = "Get current info",func=query_web,
        description="""invoke it for when you need to answer questions about current info.
             And the input should be a search query."""),
    Tool(
        name="Get current cluster state", func=get_current_cluster_state,
        description="""invoke it for when you need to get the current state in a given cluster, 
        And the input should be service name.""")
]

agent = initialize_agent(tools, llm, agent="zero-shot-react-description", max_iterations=3, verbose=True)

In [10]:
agent.run("北京今天穿什么合适？")



> Entering new AgentExecutor chain...
我需要获取当前的天气信息来回答这个问题。
Action: Get current info
Action Input: 北京今天的天气
Observation: 72°F
Thought:根据观察到的天气信息，我可以推断出适合穿什么。
Final Answer: 今天北京的天气是72°F，所以穿一件长袖T恤和一条牛仔裤应该很合适。

> Finished chain.


'今天北京的天气是72°F，所以穿一件长袖T恤和一条牛仔裤应该很合适。'

In [11]:
agent.run("如何修复的ADX集群？")



> Entering new AgentExecutor chain...
首先，我需要知道ADX集群的当前状态。
Action: Get current cluster state
Action Input: ADX
Observation: ERROR: Failed to pull image "chaocai/docker/dsp:latest
Thought:看来ADX集群的问题在于无法拉取最新的镜像。我需要查找如何解决这个问题。
Action: Get current info
Action Input: 如何修复无法拉取docker镜像的问题
Observation: 原因： 系统的时间和docker hub时间不一致,需要做系统时间与网络时间同步 · 安装时间同步 sudo yum -y install ntp ntpdate. 1 · 同步时间 sudo ntpdate cn. 更新：重启后/etc/resolv.conf的修改确实失效恢复到默认。解决办法：. 在/etc/sysconfig/network-scripts/ifcfg-eth<N> 文件中加入PEERDNS 选项。<N>可以 ... 您好，这个镜像确实是存在，我用您的命令到ssh中拉取镜像是可以拉取的，您拉取不成功，可能是因为您的网络环境和Docker Hub之间的连接不稳定或者速度较慢，或者他们在拉取时的时间Docker Hub的镜像仓库处于不可用状态，导致拉取镜像失败。 1、检查您的网络连接是否正常，并尝试使用 ... 在节点性能较差或镜像较大时，可能出现镜像无法成功下载，负载启动失败的现象。 解决方案：. (推荐)方法一：登录节点使用docker pull命令手动下拉镜像， ... 网络问题：Docker 拉取镜像需要访问Docker Hub 或其他镜像仓库，如果网络不稳定或连接不畅，可能会导致卡住。可以尝试检查网络连接，并尝试使用VPN 或更改网络环境等 ... 反馈bug/问题模板，提建议请删除1.关于你要提交的问题Q：是否搜索了issue (使用"x" 选择) [x ] 没有类似的issue 2. 详细叙述(1) 具体问题A： ... 我正在使用最简单的docker文件从另一个镜像创建docker镜像： FROM 成功构建和标记映像。但是，当我将其推送到Azure容

'修复ADX集群的方法可能包括：1. 检查网络连接，确保网络稳定；2. 同步服务器时间，确保与docker hub时间一致；3. 手动使用docker pull命令拉取镜像；4. 重新配置Docker Credential Helper来修复权限问题；5. 使用docker login命令进行身份验证，确保具备拉'

# 利用LangChain中的工具

In [7]:
pip install wolframalpha

Note: you may need to restart the kernel to use updated packages.


In [12]:
from langchain.utilities.wolfram_alpha import WolframAlphaAPIWrapper
def wolfram_api(question):
    wolfram = WolframAlphaAPIWrapper()
    return wolfram.run(question)

In [10]:
wolfram_api("What is 2x+5 = -3x + 7?")

'Assumption: 2 x + 5 = -3 x + 7 \nAnswer: x = 2/5'

In [13]:
tools = [
    Tool(
        name = "Get current info",func=query_web,
        description="""invoke it for when you need to answer questions about current info.
             And the input should be a search query."""),
    Tool(
        name="Get current cluster state", func=get_current_cluster_state,
        description="""invoke it for when you need to get the current state in a given cluster, 
        And the input should be service name."""),
    Tool(
        name="Invoke WolframAlpha API", func=wolfram_api,
        description="""invoke it for when you need to anwser or solve a mathematical question, 
        And the input is the mathematical equations""")
]

agent = initialize_agent(tools, llm, agent="zero-shot-react-description", max_iterations=10, verbose=True)

In [14]:
agent.run("在一个外星球上，这里的鸡都长着3只脚1个头，免子有4只脚1个头，一个装着鸡和免子的笼子里一共有20个头，71只脚，请问笼子里各有多少只鸡和免子？")



> Entering new AgentExecutor chain...
这是一个数学问题，我需要解决一个包含两个未知数的方程组。鸡的数量我将用x表示，免子的数量我将用y表示。根据问题，我可以得到以下两个方程：x + y = 20 (头的数量) 和 3x + 4y = 71 (脚的数量)。
Action: Invoke WolframAlpha API
Action Input: solve the equations x + y = 20 and 3x + 4y = 71
Observation: Assumption: solve x + y = 20
3 x + 4 y = 71 
Answer: x = 9 and y = 11
Thought:我现在知道最后的答案
Final Answer: 笼子里有9只鸡和11只免子。

> Finished chain.


'笼子里有9只鸡和11只免子。'